In [1]:
from pathlib import Path
import pandas as pd

# Edge mappings are only based on isomorphic graphs from the en gold train dataset
PATH = Path('../data/edge_classifier/edge_mappings_2022_05_11_18_00_36.csv')
df = pd.read_csv(PATH)

<ipython-input-1-e972a5c40aba>:6: DtypeWarning: Columns (29,30,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH)


In [2]:
df.deprel.unique()

array([nan, 'nsubj', 'obj', 'advmod', 'obl', 'obl:npmod', 'iobj', 'amod',
       'nmod:poss', 'nmod', 'nummod', 'nsubj:pass', 'nmod:tmod', 'conj',
       'expl', 'ccomp', 'advcl', 'xcomp', 'obl:tmod', 'compound',
       'nmod:npmod', 'det:predet', 'acl', 'acl:relcl', 'appos',
       'vocative', 'flat', 'parataxis'], dtype=object)

In [3]:
# Drop nodes without pos tags
df = df[(df.from_node_upos.notnull()) & (df.to_node_upos.notnull()) & (df.deprel.notnull())]

In [4]:
df

,from_node_token,from_node_Case,from_node_Number,from_node_Person,from_node_PronType,from_node_lemma,from_node_upos,from_node_xpos,to_node_token,deprel,...,to_node_Degree,to_node_Poss,to_node_NumType,to_node_Tense,to_node_VerbForm,from_node_Gender,from_node_Poss,to_node_Mood,to_node_Voice,to_node_Foreign
1,joke.v.01,NaN,NaN,NaN,NaN,joke,VERB,VBG,person.n.01,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,videotape.v.01,NaN,NaN,NaN,NaN,videotape,VERB,VBD,female.n.02,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,tackle.v.01,NaN,NaN,NaN,NaN,tackle,VERB,VBD,female.n.02,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,tackle.v.01,NaN,NaN,NaN,NaN,tackle,VERB,VBD,problem.n.02,obj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,tackle.v.01,NaN,NaN,NaN,NaN,tackle,VERB,VBD,immediately.r.01,advmod,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18690,young.a.01,NaN,NaN,NaN,NaN,young,ADJ,JJ,very.r.01,advmod,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18692,island.n.01,NaN,Sing,NaN,NaN,island,NOUN,NN,female.n.02,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18702,dig.v.02,NaN,NaN,NaN,NaN,dig,VERB,VBD,female.n.02,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18708,artist.n.01,NaN,Sing,NaN,NaN,artist,NOUN,NN,female.n.02,nsubj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def create_key(from_upos, deprel, to_upos):
    return '-'.join([from_upos, deprel, to_upos])

mapping = dict()
for _, row in df.iterrows():
    label = row.label
    key = create_key(row.from_node_upos, row.deprel, row.to_node_upos)
    if key in mapping:
        if label in mapping[key]:
            mapping[key][label] += 1
        else:
            mapping[key][label] = 1
    else:
        mapping[key] = {label: 1}

In [6]:
def filter_most_common(mapping_dict):
    filtered_dict = dict()
    for lem, count_dict in mapping_dict.items():
        if len(count_dict) == 1:
            filtered_dict[lem] = list(count_dict.keys())[0]
        else:
            filtered_dict[lem] = sorted(list(count_dict.items()), key=lambda x: x[1])[-1][0]
    return filtered_dict

In [7]:
most_common = filter_most_common(mapping)

In [8]:
import json

with open('edge_mappings_train.json', 'w') as f:
    json.dump(most_common, f, indent=2)